## Install Python Packages

In [1]:
!pip install -U dash
!pip install jupyter_dash
!pip install --upgrade plotly

     |████████████████████████████████| 9.9 MB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 36.4 MB/s ta 0:00:01
     |████████████████████████████████| 232 kB 80.7 MB/s eta 0:00:01
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.2
    Uninstalling Werkzeug-2.0.2:
      Successfully uninstalled Werkzeug-2.0.2
     |████████████████████████████████| 15.3 MB 36.8 MB/s eta 0:00:01
  Attempting uninstall: plotly
    Found existing installation: plotly 5.1.0
    Uninstalling plotly-5.1.0:
      Successfully uninstalled plotly-5.1.0


In [2]:
!pip list

Package                       Version
----------------------------- -------------------
absl-py                       0.12.0
aiohttp                       3.8.1
aiosignal                     1.2.0
ansi2html                     1.8.0
appdirs                       1.4.4
argon2-cffi                   20.1.0
astropy                       5.0
astunparse                    1.6.3
async-generator               1.10
async-timeout                 4.0.1
attrs                         21.2.0
autoai-libs                   1.13.6
autoai-ts-libs                1.1.9
autovizwidget                 0.18.0
av                            8.0.3
azure-core                    1.21.0
azure-cosmos                  4.2.0
backcall                      0.2.0
beautifulsoup4                4.10.0
biopython                     1.78
black                         19.10b0
bleach                        4.0.0
blinker                       1.4
bokeh                         2.3.3
boto3                         1.18.21
botocor

In [3]:
# Import URL browser Display Module
from IPython.display import IFrame

In [4]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px

/tmp/wsuser/ipykernel_209/2979838185.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/wsuser/ipykernel_209/2979838185.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


## Read the airline data into pandas dataframe

In [5]:
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

### Create Plotly Dash Application

In [6]:
app = JupyterDash(__name__)

### Create an application Layout using Plotly Dash and  add Drop Down

In [7]:


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                                value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

### Add Call Back Function for drop down list visualised through Pie Chart

In [8]:
# TASK 2:
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count', 
        names='class',
        title='Total Success Launched for site {entered_site}')
        return fig

##  Success Rate Per Launch Site

In [9]:
spacex_df.pivot_table('class', index='Launch Site' )

,class
Launch Site,
CCAFS LC-40,0.269231
CCAFS SLC-40,0.428571
KSC LC-39A,0.769231
VAFB SLC-4E,0.400000


## Call Back Function includes payload-slider visualised through Scatter Chart

In [10]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, payload):
    low, high = payload
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)]       
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', 
        color='Booster Version Category',
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        fig = px.scatter(filtered_df[filtered_df['Launch Site'] == entered_site],
        x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title=f'Correlation between Payload and Success for site {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, debug=True, dev_tools_hot_reload =True, threaded=True)

In [11]:
url = 'http://localhost:8090/'
IFrame(url, width=1500, height=1200)